### Today we are going to perform the simple classification of the amazon reviews' sentiment.

### Please, download the dataset amazon_baby.csv.

In [181]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.linear_model import LogisticRegression

def remove_punctuation(text):
    import string
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

baby_df = pd.read_csv('amazon_baby.csv')
baby_df.head()

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


## Exercise 1 (data preparation)
a) Remove punctuation from reviews using the given function.   
b) Replace all missing (nan) revies with empty "" string.  
c) Drop all the entries with rating = 3, as they have neutral sentiment.   
d) Set all positive ($\geq$4) ratings to 1 and negative($\leq$2) to -1.

In [182]:
#b)
baby_df["review"] = baby_df["review"].fillna("")
# a)
baby_df["review"].apply(remove_punctuation)

#short test: 
baby_df["review"][4] == 'All of my kids have cried nonstop when I tried to ween them off their pacifier until I found Thumbuddy To Loves Binky Fairy Puppet  It is an easy way to work with your kids to allow them to understand where their pacifier is going and help them part from itThis is a must buy book and a great gift for expecting parents  You will save them soo many headachesThanks for this book  You all rock'
remove_punctuation(baby_df["review"][4]) == 'All of my kids have cried nonstop when I tried to ween them off their pacifier until I found Thumbuddy To Loves Binky Fairy Puppet  It is an easy way to work with your kids to allow them to understand where their pacifier is going and help them part from itThis is a must buy book and a great gift for expecting parents  You will save them soo many headachesThanks for this book  You all rock'

True

In [183]:
#b)

#short test:
baby_df["review"][38] == baby_df["review"][38]

True

In [184]:
#c)
baby_df = baby_df.drop(baby_df[baby_df["rating"] == 3].index)
#short test:
sum(baby_df["rating"] == 3)

0

In [185]:
#d) 
baby_df["rating"] = np.where(baby_df["rating"] < 3, -1, 1)
#short test:
sum(baby_df["rating"]**2 != 1)

0

## CountVectorizer
In order to analyze strings, we need to assign them numerical values. We will use one of the simplest string representation, which transforms strings into the $n$ dimensional vectors. The number of dimensions will be the size of our dictionary, and then the values of the vector will represent the number of appereances of the given word in the sentence.

In [186]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
reviews_train_example = ["We like apples",
                   "We hate oranges",
                   "I adore bananas",
                   "We like like apples and oranges",
                   "They dislike bananas"]

X_train_example = vectorizer.fit_transform(reviews_train_example)

print(vectorizer.get_feature_names_out())
print(X_train_example.todense())



['adore' 'and' 'apples' 'bananas' 'dislike' 'hate' 'like' 'oranges' 'they'
 'we']
[[0 0 1 0 0 0 1 0 0 1]
 [0 0 0 0 0 1 0 1 0 1]
 [1 0 0 1 0 0 0 0 0 0]
 [0 1 1 0 0 0 2 1 0 1]
 [0 0 0 1 1 0 0 0 1 0]]


In [187]:
reviews_test_example = ["They like bananas",
                   "We hate oranges bananas and apples",
                   "We love bananas"] #New word!

X_test_example = vectorizer.transform(reviews_test_example)

print(X_test_example.todense())

[[0 0 0 1 0 0 1 0 1 0]
 [0 1 1 1 0 1 0 1 0 1]
 [0 0 0 1 0 0 0 0 0 1]]


We should acknowledge few facts. Firstly, CountVectorizer does not take order into account. Secondly, it ignores one-letter words (this can be changed during initialization). Finally, for test values, CountVectorizer ignores words which are not in it's dictionary.

## Exercise 2 
a) Split dataset into training and test sets.     
b) Transform reviews into vectors using CountVectorizer. 

In [188]:
#a)Split dataset into training and test sets.
train_df = baby_df.sample(frac=.8, random_state=1)
test_df = baby_df.drop(train_df.index)

In [189]:
#b)Transform reviews into vectors using CountVectorizer. 
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X_train_example = vectorizer.fit_transform(train_df.review)

# only transform on test

print(vectorizer.get_feature_names_out()[10000:10050])
# print(X_train_example.todense())
print(vectorizer.get_feature_names_out().shape)
print(X_train_example.shape) #too big to do todense()

['cassina' 'cast' 'castanets' 'casted' 'caster' 'castering' 'casters'
 'casterscons' 'castigate' 'castile' 'casting' 'castings' 'castle'
 'castles' 'castleware' 'castor' 'castors' 'castrate' 'casts' 'casual'
 'casually' 'casualties' 'casualty' 'casue' 'cat' 'catagories' 'catagory'
 'catalog' 'catalogs' 'catalogue' 'catalyst' 'catamaran' 'catapillar'
 'catapiller' 'catapult' 'catapulting' 'catapults' 'cataracts' 'catarrh'
 'catastrophe' 'catastrophes' 'catastrophic' 'catastrophically' 'catch'
 'catcher' 'catchers' 'catcherscons' 'catches' 'catching' 'catchings']
(57386,)
(133402, 57386)


## Exercise 3 
a) Train LogisticRegression model on training data (reviews processed with CountVectorizer, ratings as they were).   
b) Print 10 most positive and 10 most negative words.

In [190]:
#a)
model = LogisticRegression(max_iter=1200)
model.fit(X_train_example, train_df["rating"])


LogisticRegression(max_iter=1200)

In [191]:
#b)
model.coef_[0]
#hint: model.coef_, vectorizer.get_feature_names()

array([-6.60600431e-01, -3.30594516e-01, -1.98052713e-01, ...,
        2.88452291e-02,  2.25686977e-03, -1.59716278e-05])

## Exercise 4 
a) Predict the sentiment of test data reviews.   
b) Predict the sentiment of test data reviews in terms of probability.   
c) Find five most positive and most negative reviews.   
d) Calculate the accuracy of predictions.

In [192]:
#a)


In [193]:
#b)

#hint: model.predict_proba()

In [194]:
#c) 

#hint: use the results of b)

In [195]:
#d) 


## Exercise 5
In this exercise we will limit the dictionary of CountVectorizer to the set of significant words, defined below.


a) Redo exercises 2-5 using limited dictionary.   
b) Check the impact of all the words from the dictionary.   
c) Compare accuracy of predictions and the time of evaluation.

In [196]:
significant_words = ['love','great','easy','old','little','perfect','loves','well','able','car','broke','less','even','waste','disappointed','work','product','money','would','return']

In [197]:
#a)


In [198]:
#b)


In [199]:
#c)

#hint: %time, %timeit